# Verkeersdata Analyse Porfolio

In [1]:
# Libraries inladen
import pandas as pd

In [4]:
# Datasets inladen
accidents = pd.read_csv("datasets/accident_data.csv")
roads = pd.read_csv("datasets/road_data.csv")
weather = pd.read_csv("datasets/weather_data.csv")

### Onvolledige data verwijderen of aanvullen

In [12]:
def missende_waardes(df, name):
    print(f"\n Missende waarden in: {name}")
    print((df.isnull().sum() / len(df) * 100))

missende_waardes(accidents, "Accident Data")
missende_waardes(weather, "Weather Data")
missende_waardes(roads, "Road Data")


 Missende waarden in: Accident Data
ID               0.0000
Source           0.0000
Severity         0.0000
Start_Time       0.0000
End_Time         0.0000
Start_Lat        0.0000
Start_Lng        0.0000
End_Lat         44.0754
End_Lng         44.0754
Distance(mi)     0.0000
Description      0.0002
City             0.0038
County           0.0000
State            0.0000
Zipcode          0.0232
dtype: float64

 Missende waarden in: Weather Data
Start_Time            0.0000
Weather_Timestamp     1.5348
Temperature(F)        2.0932
Wind_Chill(F)        25.8034
Humidity(%)           2.2260
Pressure(in)          1.7856
Visibility(mi)        2.2582
Wind_Direction        2.2394
Wind_Speed(mph)       7.3974
Precipitation(in)    28.5232
Weather_Condition     2.2202
dtype: float64

 Missende waarden in: Road Data
ID                 0.0
Amenity            0.0
Bump               0.0
Crossing           0.0
Give_Way           0.0
Junction           0.0
No_Exit            0.0
Railway            0.0
R

In [ ]:
accidents.drop(['End_Lat', 'End_Lng', 'Precipitation(in)'], axis=1, inplace=True)

In [11]:
print(weather["Precipitation(in)"].head())

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Precipitation(in), dtype: float64
